# Lab06 Decision Trees

## 1 - Packages

In [91]:
import numpy as np
%matplotlib inline

## 2 - Problem Statement

Suppose you are starting a company that grows and sells wild mushrooms.

- Since not all mushrooms are edible, you'd like to be able to tell whether a given mushroom is edible or poisonous based on it's physical attributes.
- You have some existing data that you can use for this task.

Can you use the data to help you identify which mushrooms can be sold safely?

## 3 - Dataset

You will start by loading the dataset for this task. The dataset you have collected is as follows:

|                      Mushroom                      | Cap Color | Stalk Shape | Solitary | Edible |
|:--------------------------------------------------:|:---------:|:-----------:|:--------:|:------:|
| <img src="images/0.png" alt="drawing" width="50"/> |   Brown   |   Tapering  |    Yes   |    1   |
| <img src="images/1.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    Yes   |    1   |
| <img src="images/2.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |
| <img src="images/3.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |
| <img src="images/4.png" alt="drawing" width="50"/> |   Brown   |   Tapering  |    Yes   |    1   |
| <img src="images/5.png" alt="drawing" width="50"/> |    Red    |   Tapering  |    Yes   |    0   |
| <img src="images/6.png" alt="drawing" width="50"/> |    Red    |  Enlarging  |    No    |    0   |
| <img src="images/7.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    Yes   |    1   |
| <img src="images/8.png" alt="drawing" width="50"/> |    Red    |   Tapering  |    No    |    1   |
| <img src="images/9.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |

-  You have 10 examples of mushrooms. For each example, you have:
  - Three features
    - Cap Color ( `Brown` or `Red` )
    - Stalk Shape ( `Tapering` or `Enlarging` )
    - Solitary ( `Yes` or `No` )
  - Label
    - Edible ( `1` indicating yes or `0` indicating poisonous)

## 4 - One-hot Encoded Dataset

For ease of implementation, we have one-hot encoded the features (turned them into 0 or 1 valued features).

|                      Mushroom                      | Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:--------------------------------------------------:|:---------:|:--------------------:|:--------:|:------:|
| <img src="images/0.png" alt="drawing" width="50"/> |     1     |           1          |     1    |    1   |
| <img src="images/1.png" alt="drawing" width="50"/> |     1     |           0          |     1    |    1   |
| <img src="images/2.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |
| <img src="images/3.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |
| <img src="images/4.png" alt="drawing" width="50"/> |     1     |           1          |     1    |    1   |
| <img src="images/5.png" alt="drawing" width="50"/> |     0     |           1          |     1    |    0   |
| <img src="images/6.png" alt="drawing" width="50"/> |     0     |           0          |     0    |    0   |
| <img src="images/7.png" alt="drawing" width="50"/> |     1     |           0          |     1    |    1   |
| <img src="images/8.png" alt="drawing" width="50"/> |     0     |           1          |     0    |    1   |
| <img src="images/9.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |

Therefore,

- `X_train` contains three features for each example.
  - Brown Color (A value of `1` indicates "Brown" cap color and `0` indicates "Red" cap color)
  - Tapering Shape (A value of `1` indicates "Tapering Stalk Shape" and `0` indicates "Enlarging" stalk shape)
  - Solitary  (A value of `1` indicates "Yes" and `0` indicates "No")

- `y_train` is whether the mushroom is edible.
  - `y = 1` indicates edible
  - `y = 0` indicates poisonous

In [92]:
X_train = np.array([[1, 1, 1], [1, 0, 1], [1, 0, 0], [1, 0, 0], [1, 1, 1], [0, 1, 1], [0, 0, 0], [1, 0, 1], [0, 1, 0], [1, 0, 0]])
y_train = np.array([1, 1, 0, 0, 1, 0, 0, 1, 1, 0])

## 5 - Decision Tree Refresher

In this practice lab, you will build a decision tree based on the dataset provided.

- Recall that the steps for building a decision tree are as follows:
  - Start with all examples at the root node.
  - Calculate information gain for splitting on all possible features, and pick the one with the highest information gain.
  - Split dataset according to the selected feature, and create left and right branches of the tree.
  - Keep repeating splitting process until stopping criteria is met.

- In this lab, you'll implement the following functions, which will let you split a node into left and right branches using the feature with the highest information gain.
  - Calculate the entropy at a node.
  - Split the dataset at a node into left and right branches based on a given feature.
  - Calculate the information gain from splitting on a given feature.
  - Choose the feature that maximizes information gain.

- We'll then use the helper functions you've implemented to build a decision tree by repeating the splitting process until the stopping criteria is met.
  - For this lab, the stopping criteria we've chosen is setting a maximum depth of 2.

### Calculate Entropy

First, you'll write a helper function called `compute_entropy` that computes the entropy (measure of impurity) at a node.

- The function takes in a numpy array ( `y` ) that indicates whether the examples in that node are edible ( `1` ) or poisonous( `0` ).

Complete the `compute_entropy()` function below to:

- Compute $p_1$, which is the fraction of examples that are edible (i.e. have value = `1` in `y` ).
- The entropy is then calculated as

$$
H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)
$$

Notes:

- The log is calculated with base $2$.
- For implementation purposes, $0\text{log}_2(0) = 0$. That is, if `p_1 = 0` or `p_1 = 1` , set the entropy to `0` .
- Make sure to check that the data at a node is not empty (i.e. `len(y) != 0` ). Return `0` if it is.

In [93]:
def compute_entropy(y):
    entropy = 0.

    if len(y) != 0:
        p1 = len(y[y == 1]) / len(y)

        if p1 != 0 and p1 != 1:
            entropy = -p1 * np.log2(p1) - (1 - p1) * np.log2(1 - p1)
        else:
            entropy = 0.

    return entropy

### Split Dataset

Next, you'll write a helper function called `split_dataset` that takes in the data at a node and a feature to split on and splits it into left and right branches. Later in the lab, you'll implement code to calculate how good the split is.

- The function takes in the training data, the list of indices of data points at that node, along with the feature to split on.
- It splits the data and returns the subset of indices at the left and the right branch.
- For example, say we're starting at the root node (so `node_indices = [0,1,2,3,4,5,6,7,8,9]` ), and we chose to split on feature `0` , which is whether or not the example has a brown cap.
    - The output of the function is then, `left_indices = [0,1,2,3,4,7,9]` (data points with brown cap) and `right_indices = [5,6,8]` (data points without a brown cap).

| No. |                      Mushroom                      | Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:---:|:--------------------------------------------------:|:---------:|:--------------------:|:--------:|:------:|
|  0  | <img src="images/0.png" alt="drawing" width="50"/> |     1     |           1          |     1    |    1   |
|  1  | <img src="images/1.png" alt="drawing" width="50"/> |     1     |           0          |     1    |    1   |
|  2  | <img src="images/2.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |
|  3  | <img src="images/3.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |
|  4  | <img src="images/4.png" alt="drawing" width="50"/> |     1     |           1          |     1    |    1   |
|  5  | <img src="images/5.png" alt="drawing" width="50"/> |     0     |           1          |     1    |    0   |
|  6  | <img src="images/6.png" alt="drawing" width="50"/> |     0     |           0          |     0    |    0   |
|  7  | <img src="images/7.png" alt="drawing" width="50"/> |     1     |           0          |     1    |    1   |
|  8  | <img src="images/8.png" alt="drawing" width="50"/> |     0     |           1          |     0    |    1   |
|  9  | <img src="images/9.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |

Please complete the `split_dataset()` function shown below.

- For each index in `node_indices` :
  - If the value of `X` at that index for that feature is `1`, add the index to `left_indices` .
  - If the value of `X` at that index for that feature is `0`, add the index to `right_indices` .

In [94]:
def split_dataset(X, node_indices, feature):
    left_indices = []
    right_indices = []

    for i in node_indices:
        if X[i][feature] == 1:
            left_indices.append(i)
        else:
            right_indices.append(i)

    return left_indices, right_indices

### Calculate Information Gain

Next, you'll write a function called `information_gain` that takes in the training data, the indices at a node and a feature to split on and returns the information gain from the split.

Please complete the `compute_information_gain()` function shown below to compute

$$
\text{Information Gain} = H(p_1^\text{node})- (w^{\text{left}}H(p_1^\text{left}) + w^{\text{right}}H(p_1^\text{right}))
$$

where

- $H(p_1^\text{node})$ is entropy at the node.
- $H(p_1^\text{left})$ and $H(p_1^\text{right})$ are the entropies at the left and the right branches resulting from the split.
- $w^{\text{left}}$ and $w^{\text{right}}$ are the proportion of examples at the left and right branch, respectively.

Notes:

- You can use the `compute_entropy()` function that you implemented above to calculate the entropy.
- We've provided some starter code that uses the `split_dataset()` function you implemented above to split the dataset.

In [95]:
def compute_information_gain(X, y, node_indices, feature):
    left_indices, right_indices = split_dataset(X, node_indices, feature)

    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]

    information_gain = 0

    node_entropy = compute_entropy(y_node)
    left_entropy = compute_entropy(y_left)
    right_entropy = compute_entropy(y_right)

    w_left = len(X_left) / len(X_node)
    w_right = len(X_right) / len(X_node)

    information_gain = node_entropy - (w_left * left_entropy + w_right * right_entropy)

    return information_gain

### Get Best Split

Now let's write a function to get the best feature to split on by computing the information gain from each feature as we did above and returning the feature that gives the maximum information gain.

Please complete the `get_best_split()` function shown below.

- The function takes in the training data, along with the indices of datapoint at that node.
- The output of the function is the feature that gives the maximum information gain.
  - You can use the `compute_information_gain()` function to iterate through the features and calculate the information for each feature.

In [96]:
def get_best_split(X, y, node_indices):
    num_features = X.shape[1]
    best_feature = -1
    max_info_gain = 0

    for feature in range(num_features):
        info_gain = compute_information_gain(X, y, node_indices, feature)

        if info_gain > max_info_gain:
            max_info_gain = info_gain
            best_feature = feature

    return best_feature

## 6 - Building the Tree

In [97]:
def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    if current_depth == max_depth:
        formatting = " " * current_depth + "-" * current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return

    best_feature = get_best_split(X, y, node_indices)

    formatting = "-" * current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))

    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))

    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth + 1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth + 1)


tree = []
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)

 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]
